In [ ]:
from mwis_solution import GCN, train_gcn, mis_search
from data_creation import create_er_dataset
from dgl.dataloading import GraphDataLoader
import torch

In [ ]:
# Create some sample problems
train_data = create_er_dataset(200, 50, solver='networkx', weighted=True)
evaluation_data = create_er_dataset(1, 150, solver='networkx', weighted=True)

# Fit a graph neural network to these problems
net = GCN(in_feats=2, hidden_size=32)
optimizer = torch.optim.AdamW(net.parameters(), lr=5e-3)

dataset_size = len(train_data)
gdl = GraphDataLoader([train_data[x]['dgl_graph'] for x in range(int(dataset_size*0.8))], batch_size=80)
gdl_te = GraphDataLoader([train_data[x]['dgl_graph'] for x in range(int(dataset_size*0.8),dataset_size)], 
                         batch_size=min(int(dataset_size*0.2),100))

train_gcn(net, gdl, gdl_te, optimizer, 200)

In [ ]:
# Evaluate on a new graph
evaluation_graph = evaluation_data[0]['dgl_graph']
print(f"Previous MWIS: {evaluation_data[0]['mwis']}")
max_mwis, sol, _ = mis_search(evaluation_graph, net, time_limit=10.0, passes=2, verbose=True, p_best=6)

# Check the solution and it's feasibility
print(f"MWIS: {max_mwis}, feasibility {sol.check_solution_feasibility()}")